In [1]:
import boto3 
import json
import time

In [7]:
CLIENT = boto3.client("athena")

DATA_CATALOG = "AwsDataCatalog"
DATABASE_NAME = "abraseldatalake_v1"
RESULT_OUTPUT_LOCATION = "s3://abrasel-datalake/query-results/"
TABLE_NAME = "cnpj_ativos"

In [5]:
def has_query_succeeded(execution_id):
    state = "RUNNING"
    max_execution = 5

    while max_execution > 0 and state in ["RUNNING", "QUEUED"]:
        max_execution -= 1
        response = CLIENT.get_query_execution(QueryExecutionId=execution_id)
        if (
            "QueryExecution" in response
            and "Status" in response["QueryExecution"]
            and "State" in response["QueryExecution"]["Status"]
        ):
            state = response["QueryExecution"]["Status"]["State"]
            if state == "SUCCEEDED":
                return True

        time.sleep(30)

    return False

In [8]:
def get_num_rows():
    query = f"SELECT COUNT(*) from {DATA_CATALOG}.{DATABASE_NAME}.{TABLE_NAME}"
    response = CLIENT.start_query_execution(
        QueryString=query,
        ResultConfiguration={"OutputLocation": RESULT_OUTPUT_LOCATION}
    )

    return response["QueryExecutionId"]


In [9]:
def get_query_results(execution_id):
    response = CLIENT.get_query_results(
        QueryExecutionId=execution_id
    )

    results = response['ResultSet']['Rows']
    return results

In [12]:
id_consulta = get_num_rows()


In [14]:
print(get_query_results(id_consulta))

[{'Data': [{'VarCharValue': '_col0'}]}, {'Data': [{'VarCharValue': '67665840'}]}]


In [5]:
linhas = 0
with open(r"C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\ESTABELECIMENTOSCSV\K3241.K03200Y0.D30114.csv", mode='r', encoding='ISO-8859-1', errors='ignore') as arq:
    #arq.seek(0, 2)
    #print(arq.tell())
    #print(len(arq.readline()))
    #linhas = arq.tell() // len(arq.readline())
    for linha in arq:
        linhas += 1
linhas

12374347

In [ ]:
import requests
import pandas as pd
import json

df = pd.read_csv(r"C:\Users\ABRASEL NACIONAL\Downloads\cnaes.csv")
df.rename(columns={'Id':'Codigo Cnae'}, inplace=True)
df

,Codigo Cnae,Descrição,Código Longo
0,111301,CULTIVO DE ARROZ,0111-3/01
1,111302,CULTIVO DE MILHO,0111-3/02
2,111303,CULTIVO DE TRIGO,0111-3/03
3,111399,CULTIVO DE OUTROS CEREAIS NÃO ESPECIFICADOS AN...,0111-3/99
4,112101,CULTIVO DE ALGODÃO HERBÁCEO,0112-1/01
...,...,...,...
495,2920402,FABRICAÇÃO DE MOTORES PARA CAMINHÕES E ÔNIBUS,2920-4/02
496,2930101,"FABRICAÇÃO DE CABINES, CARROCERIAS E REBOQUES ...",2930-1/01
497,2930102,FABRICAÇÃO DE CARROCERIAS PARA ÔNIBUS,2930-1/02
498,2930103,"FABRICAÇÃO DE CABINES, CARROCERIAS E REBOQUES ...",2930-1/03


In [8]:
linhas = 0
diretorio =r"C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\ESTABELECIMENTOSCSV\K3241.K03200Y0.D30114.csv"

with open(diretorio, mode='r', encoding='ISO-8859-1', errors='ignore') as file:
    for linha in file:
        linhas +=1
print(linhas)

12374347


In [11]:
import pandas as pd
dados = pd.read_csv(diretorio, sep=';', encoding='ISO-8859-1', skiprows=12374327, nrows=15)
dados

,49186981,0001,00,1,JUEMPREENDIMENTOS,02,20230114,00.1,Unnamed: 8,Unnamed: 9,...,5155,31,75960880,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,JULIANALUIZA1960@GMAIL.COM,Unnamed: 28,Unnamed: 29
0,49186993,1,27,1,JS PESCA E CAMPING,2,20230114,0,NaN,NaN,...,4961,34,91032522,NaN,NaN,NaN,NaN,JOEDESSILVA3@GMAIL.COM,NaN,NaN
1,49187003,1,75,1,CAROL DOCES E CURSOS,2,20230114,0,NaN,NaN,...,9373,62,93620026,NaN,NaN,NaN,NaN,CAROL090908@GMAIL.COM,NaN,NaN
2,49187014,1,55,1,IVANILSON CONSTRUCOES,2,20230114,0,NaN,NaN,...,6887,19,99616543,NaN,NaN,NaN,NaN,TILIN_CORUJA@HOTMAIL.COM,NaN,NaN
3,49187026,1,80,1,RMFBIKE,2,20230114,0,NaN,NaN,...,5819,22,99224482,NaN,NaN,NaN,NaN,MEDRMF@GMAIL.COM,NaN,NaN
4,49187037,1,60,1,ROBUR TELECOM,2,20230114,0,NaN,NaN,...,6377,11,81185811,NaN,NaN,NaN,NaN,JFLIMA@R7.COM,NaN,NaN
5,49187048,1,40,1,RAFAELA FERREIRA,2,20230114,0,NaN,NaN,...,7107,11,0,NaN,NaN,NaN,NaN,RAFAELA.DUPLESSIS@HOTMAIL.COM,NaN,NaN
6,49187058,1,85,1,NEVES MOVEIS E ARTEFATOS DE MADEIRAS,2,20230114,0,NaN,NaN,...,9865,66,81067675,NaN,NaN,NaN,NaN,DEUSMAR.NEVES@HOTMAIL.COM,NaN,NaN
7,49187069,1,65,1,RICHARDES CAULA GESTAO EM SAUDE,2,20230114,0,NaN,NaN,...,2531,81,30717548,NaN,NaN,NaN,NaN,RICHARDES1@HOTMAIL.COM,NaN,NaN
8,49187080,1,25,1,SAPLAK EXPRESS,2,20230114,0,NaN,NaN,...,7407,41,91124973,NaN,NaN,NaN,NaN,MATHEUS-SAPLAK-DE-ANDRADE@HOTMAIL.COM,NaN,NaN
9,49187091,1,5,1,CRIS NASSIF CROCHES,2,20230114,0,NaN,NaN,...,8841,55,32262581,NaN,NaN,NaN,NaN,CRISTINES21@HOTMAIL.COM,NaN,NaN
